Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/automated-machine-learning/forecasting-hierarchical-timeseries/auto-ml-forecasting-hierarchical-timeseries.png)

# Hierarchical Time Series - Automated ML
**_Generate hierarchical time series forecasts with Automated Machine Learning_**

---

For this notebook we are using a synthetic dataset portraying sales data to predict the the quantity of a vartiety of product skus across several states, stores, and product categories.

**NOTE: There are limits on how many runs we can do in parallel per workspace, and we currently recommend to set the parallelism to maximum of 320 runs per experiment per workspace. If users want to have more parallelism and increase this limit they might encounter Too Many Requests errors (HTTP 429).**

### Prerequisites
You'll need to create a compute Instance by following the instructions in the [EnvironmentSetup.md](../Setup_Resources/EnvironmentSetup.md).

## 1.0 Set up workspace, datastore, experiment

In [ ]:
import azureml.core
from azureml.core import Workspace, Datastore
import pandas as pd

# Set up your workspace
ws = Workspace.from_config()
ws.get_details()

# Set up your datastores
dstore = ws.get_default_datastore()

output = {}
output["SDK version"] = azureml.core.VERSION
output["Subscription ID"] = ws.subscription_id
output["Workspace"] = ws.name
output["Resource Group"] = ws.resource_group
output["Location"] = ws.location
output["Default datastore name"] = dstore.name
output["SDK Version"] = azureml.core.VERSION
pd.set_option("display.max_colwidth", None)
outputDf = pd.DataFrame(data=output, index=[""])
outputDf.T

### Choose an experiment

In [ ]:
from azureml.core import Experiment

experiment = Experiment(ws, "automl-hts")

print("Experiment name: " + experiment.name)

## 2.0 Data


### Upload local csv files to datastore
You can upload your train and inference csv files to the default datastore in your workspace. 

A Datastore is a place where data can be stored that is then made accessible to a compute either by means of mounting or copying the data to the compute target.
Please refer to [Datastore](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.datastore.datastore?view=azure-ml-py) documentation on how to access data from Datastore.

In [ ]:
datastore_path = "hts-sample"

In [ ]:
datastore = ws.get_default_datastore()
datastore

### Create the TabularDatasets 

Datasets in Azure Machine Learning are references to specific data in a Datastore. The data can be retrieved as a [TabularDatasets](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.tabulardataset?view=azure-ml-py). We will read in the data as a pandas DataFrame, upload to the data store and register them to your Workspace using ```register_pandas_dataframe``` so they can be called as an input into the training pipeline. We will use the inference dataset as part of the forecasting pipeline. The step need only be completed once.

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

registered_train = TabularDatasetFactory.register_pandas_dataframe(
    pd.read_csv("Data/hts-sample-train.csv"),
    target=(datastore, "hts-sample"),
    name="hts-sales-train",
)
registered_inference = TabularDatasetFactory.register_pandas_dataframe(
    pd.read_csv("Data/hts-sample-test.csv"),
    target=(datastore, "hts-sample"),
    name="hts-sales-test",
)

## 3.0 Build the training pipeline
Now that the dataset, WorkSpace, and datastore are set up, we can put together a pipeline for training.

> Note that if you have an AzureML Data Scientist role, you will not have permission to create compute resources. Talk to your workspace or IT admin to create the compute targets described in this section, if they do not already exist.

### Choose a compute target

You will need to create a [compute target](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-set-up-training-targets#amlcompute) for your AutoML run. In this tutorial, you create AmlCompute as your training compute resource.

\*\*Creation of AmlCompute takes approximately 5 minutes.**

If the AmlCompute with that name is already in your workspace this code will skip the creation process. As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read this [article](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-manage-quotas) on the default limits and how to request more quota.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# Name your cluster
compute_name = "hts-compute"


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print("Found compute target: " + compute_name)
else:
    print("Creating a new compute target...")
    provisioning_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_D16S_V3", max_nodes=20
    )
    # Create the compute target
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

    # Can poll for a minimum number of nodes and for a specific timeout.
    # If no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(
        show_output=True, min_node_count=None, timeout_in_minutes=20
    )

    # For a more detailed view of current cluster status, use the 'status' property
    print(compute_target.status.serialize())

### Set up training parameters

This dictionary defines the AutoML and hierarchy settings. For this forecasting task we need to define several settings inncluding the name of the time column, the maximum forecast horizon, the hierarchy definition, and the level of the hierarchy at which to train.

| Property                           | Description|
| :---------------                   | :------------------- |
| **task**                           | forecasting |
| **primary_metric**                 | This is the metric that you want to optimize.<br> Forecasting supports the following primary metrics <br><i>spearman_correlation</i><br><i>normalized_root_mean_squared_error</i><br><i>r2_score</i><br><i>normalized_mean_absolute_error</i> |
| **blocked_models**                 | Blocked models won't be used by AutoML. |
| **iteration_timeout_minutes**      | Maximum amount of time in minutes that the model can train. This is optional but provides customers with greater control on exit criteria. |
| **iterations**                     | Number of models to train. This is optional but provides customers with greater control on exit criteria. |
| **experiment_timeout_hours**       | Maximum amount of time in hours that the experiment can take before it terminates. This is optional but provides customers with greater control on exit criteria. |
| **label_column_name**              | The name of the label column. |
| **forecast_horizon**               | The forecast horizon is how many periods forward you would like to forecast. This integer horizon is in units of the timeseries frequency (e.g. daily, weekly). Periods are inferred from your data. |
| **n_cross_validations**            | Number of cross validation splits. Rolling Origin Validation is used to split time-series in a temporally consistent way. |
| **enable_early_stopping**          | Flag to enable early termination if the score is not improving in the short term. |
| **time_column_name**               | The name of your time column. |
| **hierarchy_column_names**         | The names of columns that define the hierarchical structure of the data from highest level to most granular. |
| **training_level**                 | The level of the hierarchy to be used for training models. |
| **enable_engineered_explanations** | Engineered feature explanations will be downloaded if enable_engineered_explanations flag is set to True. By default it is set to False to save storage space. |
| **time_series_id_column_name**     | The column names used to uniquely identify timeseries in data that has multiple rows with the same timestamp. |
| **track_child_runs**               | Flag to disable tracking of child runs. Only best run is tracked if the flag is set to False (this includes the model and metrics of the run). |
| **pipeline_fetch_max_batch_size**  | Determines how many pipelines (training algorithms) to fetch at a time for training, this helps reduce throttling when training at large scale. |
| **model_explainability**           | Flag to disable explaining the best automated ML model at the end of all training iterations. The default is True and will block non-explainable models which may impact the forecast accuracy. For more information, see [Interpretability: model explanations in automated machine learning](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-machine-learning-interpretability-automl). |

In [ ]:
from azureml.train.automl.runtime._hts.hts_parameters import HTSTrainParameters

model_explainability = True

engineered_explanations = False
# Define your hierarchy. Adjust the settings below based on your dataset.
hierarchy = ["state", "store_id", "product_category", "SKU"]
training_level = "SKU"

# Set your forecast parameters. Adjust the settings below based on your dataset.
time_column_name = "date"
label_column_name = "quantity"
forecast_horizon = 7


automl_settings = {
    "task": "forecasting",
    "primary_metric": "normalized_root_mean_squared_error",
    "label_column_name": label_column_name,
    "time_column_name": time_column_name,
    "forecast_horizon": forecast_horizon,
    "hierarchy_column_names": hierarchy,
    "hierarchy_training_level": training_level,
    "track_child_runs": False,
    "pipeline_fetch_max_batch_size": 15,
    "model_explainability": model_explainability,
    # The following settings are specific to this sample and should be adjusted according to your own needs.
    "iteration_timeout_minutes": 10,
    "iterations": 10,
    "n_cross_validations": 2,
}

hts_parameters = HTSTrainParameters(
    automl_settings=automl_settings,
    hierarchy_column_names=hierarchy,
    training_level=training_level,
    enable_engineered_explanations=engineered_explanations,
)

### Set up hierarchy training pipeline

Parallel run step is leveraged to train the hierarchy. To configure the ParallelRunConfig you will need to determine the appropriate number of workers and nodes for your use case. The `process_count_per_node` is based off the number of cores of the compute VM. The node_count will determine the number of master nodes to use, increasing the node count will speed up the training process.

* **experiment:** The experiment used for training.
* **train_data:** The tabular dataset to be used as input to the training run.
* **node_count:** The number of compute nodes to be used for running the user script. We recommend to start with 3 and increase the node_count if the training time is taking too long.
* **process_count_per_node:** Process count per node, we recommend 2:1 ratio for number of cores: number of processes per node. eg. If node has 16 cores then configure 8 or less process count per node or optimal performance.
* **train_pipeline_parameters:** The set of configuration parameters defined in the previous section. 

Calling this method will create a new aggregated dataset which is generated dynamically on pipeline execution.

In [ ]:
from azureml.contrib.automl.pipeline.steps import AutoMLPipelineBuilder


training_pipeline_steps = AutoMLPipelineBuilder.get_many_models_train_steps(
    experiment=experiment,
    train_data=registered_train,
    compute_target=compute_target,
    node_count=2,
    process_count_per_node=8,
    train_pipeline_parameters=hts_parameters,
)

In [ ]:
from azureml.pipeline.core import Pipeline

training_pipeline = Pipeline(ws, steps=training_pipeline_steps)

### Submit the pipeline to run
Next we submit our pipeline to run. The whole training pipeline takes about 1h using a Standard_D16_V3 VM with our current ParallelRunConfig setting.

In [ ]:
training_run = experiment.submit(training_pipeline)

In [ ]:
training_run.wait_for_completion(show_output=False)

Check the run status, if training_run is in completed state, continue to forecasting. If training_run is in another state, check the portal for failures.

### [Optional] Get the explanations
First we need to download the explanations to the local disk.

In [ ]:
if model_explainability:
    expl_output = training_run.get_pipeline_output("explanations")
    expl_output.download("training_explanations")
else:
    print(
        "Model explanations are available only if model_explainability is set to True."
    )

The explanations are downloaded to the "training_explanations/azureml" directory.

In [ ]:
import os

if model_explainability:
    explanations_dirrectory = os.listdir(
        os.path.join("training_explanations", "azureml")
    )
    if len(explanations_dirrectory) > 1:
        print(
            "Warning! The directory contains multiple explanations, only the first one will be displayed."
        )
    print("The explanations are located at {}.".format(explanations_dirrectory[0]))
    # Now we will list all the explanations.
    explanation_path = os.path.join(
        "training_explanations",
        "azureml",
        explanations_dirrectory[0],
        "training_explanations",
    )
    print("Available explanations")
    print("==============================")
    print("\n".join(os.listdir(explanation_path)))
else:
    print(
        "Model explanations are available only if model_explainability is set to True."
    )

View the explanations on "state" level.

In [ ]:
from IPython.display import display

explanation_type = "raw"
level = "state"

if model_explainability:
    display(
        pd.read_csv(
            os.path.join(explanation_path, "{}_explanations_{}.csv").format(
                explanation_type, level
            )
        )
    )

## 5.0 Forecasting
For hierarchical forecasting we need to provide the HTSInferenceParameters object.
#### HTSInferenceParameters arguments
* **hierarchy_forecast_level:** The default level of the hierarchy to produce prediction/forecast on.
* **allocation_method:** \[Optional] The disaggregation method to use if the hierarchy forecast level specified is below the define hierarchy training level. <br><i>(average historical proportions) 'average_historical_proportions'</i><br><i>(proportions of the historical averages) 'proportions_of_historical_average'</i>

#### get_many_models_batch_inference_steps arguments
* **experiment:** The experiment used for inference run.
* **inference_data:** The data to use for inferencing. It should be the same schema as used for training.
* **compute_target:** The compute target that runs the inference pipeline.
* **node_count:** The number of compute nodes to be used for running the user script. We recommend to start with the number of cores per node (varies by compute sku).
* **process_count_per_node:** The number of processes per node.
* **train_run_id:** \[Optional] The run id of the hierarchy training, by default it is the latest successful training hts run in the experiment.
* **train_experiment_name:** \[Optional] The train experiment that contains the train pipeline. This one is only needed when the train pipeline is not in the same experiement as the inference pipeline.
* **process_count_per_node:** \[Optional] The number of processes per node, by default it's 4.

In [ ]:
from azureml.train.automl.runtime._hts.hts_parameters import HTSInferenceParameters

inference_parameters = HTSInferenceParameters(
    hierarchy_forecast_level="store_id",  # The setting is specific to this dataset and should be changed based on your dataset.
    allocation_method="proportions_of_historical_average",
)

steps = AutoMLPipelineBuilder.get_many_models_batch_inference_steps(
    experiment=experiment,
    inference_data=registered_inference,
    compute_target=compute_target,
    inference_pipeline_parameters=inference_parameters,
    node_count=2,
    process_count_per_node=8,
)

In [ ]:
from azureml.pipeline.core import Pipeline

inference_pipeline = Pipeline(ws, steps=steps)

In [ ]:
inference_run = experiment.submit(inference_pipeline)
inference_run.wait_for_completion(show_output=False)

## Retrieve results

Forecast results can be retrieved through the following code. The prediction results summary and the actual predictions are downloaded in forecast_results folder

In [ ]:
forecasts = inference_run.get_pipeline_output("forecasts")
forecasts.download("forecast_results")

## Resbumit the Pipeline

The inference pipeline can be submitted with different configurations.

In [ ]:
inference_run = experiment.submit(
    inference_pipeline, pipeline_parameters={"hierarchy_forecast_level": "state"}
)
inference_run.wait_for_completion(show_output=False)